In [38]:
import openai
import datasets
import pandas as pd

from fp_dataset_artifacts.utils import init_openai, get_qa_response
from fp_dataset_artifacts.squad import INTRO
from datasets import list_datasets, load_dataset, list_metrics, load_metric, concatenate_datasets, Dataset

init_openai()

data = load_dataset('squad')
data

Reusing dataset squad (/home/x/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [5]:
train = data['train']
valid = data['validation']

In [6]:
# Get counts for each title and count of unique titles
train.remove_columns(['context', 'question', 'answers']).to_pandas().title.value_counts()

New_York_City            817
American_Idol            802
Beyoncé                  758
Frédéric_Chopin          697
Queen_Victoria           680
                        ... 
Great_Plains              47
Tristan_da_Cunha          44
Pitch_(music)             36
Matter                    24
Myocardial_infarction     22
Name: title, Length: 442, dtype: int64

In [7]:
# Get counts for each title and count of unique titles
valid.remove_columns(['context', 'question', 'answers']).to_pandas().title.value_counts()

Super_Bowl_50                                810
Nikola_Tesla                                 511
Martin_Luther                                479
American_Broadcasting_Company                407
Doctor_Who                                   319
Economic_inequality                          297
Victoria_and_Albert_Museum                   295
Chloroplast                                  295
Rhine                                        291
Teacher                                      268
Newcastle_upon_Tyne                          264
Oxygen                                       247
Warsaw                                       247
Genghis_Khan                                 245
European_Union_law                           243
Apollo_program                               242
Immune_system                                238
Kenya                                        231
Yuan_dynasty                                 228
Huguenot                                     217
Steam_engine        

In [8]:
# Make examples without context
valid[200]

{'id': '56beb3083aeaaa14008c923e',
 'title': 'Super_Bowl_50',
 'context': 'Despite waiving longtime running back DeAngelo Williams and losing top wide receiver Kelvin Benjamin to a torn ACL in the preseason, the Carolina Panthers had their best regular season in franchise history, becoming the seventh team to win at least 15 regular season games since the league expanded to a 16-game schedule in 1978. Carolina started the season 14–0, not only setting franchise records for the best start and the longest single-season winning streak, but also posting the best start to a season by an NFC team in NFL history, breaking the 13–0 record previously shared with the 2009 New Orleans Saints and the 2011 Green Bay Packers. With their NFC-best 15–1 regular season record, the Panthers clinched home-field advantage throughout the NFC playoffs for the first time in franchise history. Ten players were selected to the Pro Bowl (the most in franchise history) along with eight All-Pro selections.',
 'que

In [52]:
# Prep examples and prompts for evaluation
def map_example(x):
    question = x['question']
    answer = x['answers']['text'][0]
    context = x['context']
    return {'example': f'{context}\nQ: {question}\nA: {answer}'}

def map_query(x):
    question = x['question']
    context = x['context']
    return {'query': f'{context}\nQ: {question}\nA: '}
    
print(map_example(valid[123])['example'])
print(map_query(valid[123])['query'])

The league eventually narrowed the bids to three sites: New Orleans' Mercedes-Benz Superdome, Miami's Sun Life Stadium, and the San Francisco Bay Area's Levi's Stadium.
Q: What was the name of New Orleans' superdome at the time that Super Bowl 50 took place?
A: Mercedes-Benz Superdome
The league eventually narrowed the bids to three sites: New Orleans' Mercedes-Benz Superdome, Miami's Sun Life Stadium, and the San Francisco Bay Area's Levi's Stadium.
Q: What was the name of New Orleans' superdome at the time that Super Bowl 50 took place?
A: 


In [53]:
train = train.map(map_example)

  0%|          | 0/87599 [00:00<?, ?ex/s]

In [54]:
valid = valid.map(map_query)

  0%|          | 0/10570 [00:00<?, ?ex/s]

In [55]:
train = train.shuffle(0)

In [66]:
n_examples = 5
n_samples = 10

prompts = []
for i in range(n_samples):
    examples = train[i:i+n_examples]
    examples = '\n\n'.join(examples['example'])
    query = valid[i]['query']
    
    prompt = f"""{INTRO}\n\n{examples}\n\n{query}"""
    answers = valid['answers'][0]['text']
    prompts.append({'prompt': prompt, 'answers': answers})

In [67]:
sample_data = Dataset.from_pandas(pd.DataFrame(prompts))
sample_data

Dataset({
    features: ['prompt', 'answers'],
    num_rows: 10
})

In [68]:
print(sample_data[0]['prompt'])

I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will give you the answer.

Certain technological inventions of the period – whether of Arab or Chinese origin, or unique European innovations – were to have great influence on political and social developments, in particular gunpowder, the printing press and the compass. The introduction of gunpowder to the field of battle affected not only military organisation, but helped advance the nation state. Gutenberg's movable type printing press made possible not only the Reformation, but also a dissemination of knowledge that would lead to a gradually more egalitarian society. The compass, along with other innovations such as the cross-staff, the mariner's astrolabe, and advances in shipbuilding, enabled the navigation of the World Oceans, and the early phases of colonialism. Other inventions had a greater impact on everyday life, such as eyeglasses and the weight-driven clock.
Q: Which inve

In [65]:
get_qa_response(sample_data[0]['prompt'])

<OpenAIObject text_completion id=cmpl-4CE9hxERmlseR5kD3GAXpA1RIRAE5 at 0x7f5e016e84a0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1638819697,
  "id": "cmpl-4CE9hxERmlseR5kD3GAXpA1RIRAE5",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}

In [45]:
get_qa_response(sample_data[1]['prompt'])

<OpenAIObject text_completion id=cmpl-4CE400RiseZxFUzlUzqgzUr16uwc5 at 0x7f5e882f79f0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1638819344,
  "id": "cmpl-4CE400RiseZxFUzlUzqgzUr16uwc5",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}

In [46]:
get_qa_response(sample_data[2]['prompt'])

<OpenAIObject text_completion id=cmpl-4CE4fTgI7bME57BzwcDPDDYteBXCX at 0x7f5e2c124540> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1638819385,
  "id": "cmpl-4CE4fTgI7bME57BzwcDPDDYteBXCX",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}

In [60]:
def get_qa_response(prompt, engine='curie'):
    response = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        temperature=0.9,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=['\n'],
    )
    return response

In [61]:
get_qa_response(sample_data[2]['prompt'])

<OpenAIObject text_completion id=cmpl-4CE9OqmS7DJ6RkTGxAq2bYUPzB3lx at 0x7f5e01668f90> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1638819678,
  "id": "cmpl-4CE9OqmS7DJ6RkTGxAq2bYUPzB3lx",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}

In [49]:
get_qa_response(sample_data[2]['prompt'], 'davinci')

<OpenAIObject text_completion id=cmpl-4CE71tI6u7UZOp70N4fe931BlU4gj at 0x7f5e015a8f40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1638819531,
  "id": "cmpl-4CE71tI6u7UZOp70N4fe931BlU4gj",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}